In [1]:
import pandas as pd

In [3]:
parquet_path = "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/2wikimqa_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet"
df = pd.read_parquet(parquet_path)
df.head()

,index,context,length,distract_questions,data_source,prompt,ability,reward_model,extra_info
0,2500,Passage 1:\nRingo-en no shōjo\nThe art directo...,14326,[Where did the director of film Butley (Film) ...,custom_longcontext_needle_qa_2wikimqa_qwen_fil...,[{'content': 'Please read the following text. ...,longcontext_qa,"{'ground_truth': ['Wedding Bills'], 'style': '...","{'index': 0, 'input_question': 'Which film who..."
1,2501,Passage 1:\nPeter Levin\nPeter Levin is an Ame...,14146,[When did the director of film The Yellow Tick...,custom_longcontext_needle_qa_2wikimqa_qwen_fil...,[{'content': 'Please read the following text. ...,longcontext_qa,"{'ground_truth': ['25 May 2011'], 'style': 'ru...","{'index': 1, 'input_question': 'When did the d..."
2,2502,"Passage 1:\nYulia Meltzer\nYudith"" Yulia"" Isaa...",14086,"[Are both villages, Ezz Od Din and Ișnovăț, lo...",custom_longcontext_needle_qa_2wikimqa_qwen_fil...,[{'content': 'Please read the following text. ...,longcontext_qa,"{'ground_truth': ['14 April 1943'], 'style': '...","{'index': 2, 'input_question': 'What is the da..."
3,2503,Passage 1:\nJohn McMahon (Surrey and Somerset ...,14505,[When did the director of film Our Mrs. Mcches...,custom_longcontext_needle_qa_2wikimqa_qwen_fil...,[{'content': 'Please read the following text. ...,longcontext_qa,"{'ground_truth': ['Harrison Eiteljorg'], 'styl...","{'index': 3, 'input_question': 'Who was born f..."
4,2504,Passage 1:\nRufus Norris\nRufus Norris( born 1...,14193,"[Which film was released first, Weird Woman or...",custom_longcontext_needle_qa_2wikimqa_qwen_fil...,[{'content': 'Please read the following text. ...,longcontext_qa,"{'ground_truth': ['Barbados Quest'], 'style': ...","{'index': 4, 'input_question': 'Which film has..."


In [6]:
df['prompt'][0][0]

{'content': 'Please read the following text.\nPassage 1:\nRingo-en no shōjo\nThe art director was Tomoo Shimogawara.\n\nPassage 2:\n{"bec275d3-24f0-453c-bc5b-1a10f22d761b": "Do both films: Jackie, Ethel, Joan: The Women Of Camelot and 12:01 (1993 Film) have the directors from the same country?"}.Patrice Chéreau\nPatrice Chéreau (2 November 1944 – 7 October 2013) was a French opera and theatre director, filmmaker, actor and producer.In France he is best known for his work for the theatre, internationally for his films "La Reine Margot" and "Intimacy", and for his staging of the "Jahrhundertring", the centenary "Ring Cycle" at the Bayreuth Festival in 1976.Winner of almost twenty movie awards, including the Cannes Jury Prize and the Golden Berlin Bear, Chéreau served as president of the jury at the 2003 Cannes festival.From 1966, he was artistic director of the "Public-Theatre" in the Parisian suburb of Sartrouville, where in his team were stage designer Richard Peduzzi, costume designer

In [14]:
from pprint import pprint
import pandas as pd
from transformers import AutoTokenizer

# 预先加载 tokenizer（避免每次调用都重新下载）
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

def analyze_prompt_tokens(parquet_path, column="prompt", key="content"):
    """
    统计 parquet 文件中指定列的文本 token 长度 (使用 Qwen2.5-7B-Instruct tokenizer)

    参数:
        parquet_path (str): parquet 文件路径
        column (str): 包含字典的列名 (默认 "prompt")
        key (str): 需要提取的 key (默认 "content")

    返回:
        dict: { "max": 最大长度, "min": 最小长度, "avg": 平均长度 }
    """
    df = pd.read_parquet(parquet_path)
    print(len(df))

    # 提取并统计 token 长度
    def get_token_length(row):
        content = row[column][0][key]
        tokens = tokenizer.encode(content, add_special_tokens=False)
        return len(tokens)

    df["token_length"] = df.apply(get_token_length, axis=1)

    return {
        "max": int(df["token_length"].max()),
        "min": int(df["token_length"].min()),
        "avg": float(df["token_length"].mean())
    }

# 示例调用
# parquet_path = "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/2wikimqa_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet"
# stats = analyze_prompt_tokens(parquet_path)
parquet_list = [
    "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/hotpotqa_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet",
    "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/musique_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet",
    "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/2wikimqa_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet",
]
for parquet_path in parquet_list:
    stats = analyze_prompt_tokens(parquet_path)
    pprint(f"{parquet_path} -> {stats}")

2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/hotpotqa_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet '
 "-> {'max': 20670, 'min': 16272, 'avg': 18607.62}")
2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/musique_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet '
 "-> {'max': 20623, 'min': 16495, 'avg': 19411.4856}")
2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/2wikimqa_filtered_distractor_256_start_idx2500_end_idx5000_seq16384/train.parquet '
 "-> {'max': 20576, 'min': 14911, 'avg': 18588.0688}")


In [13]:
no_dis_parquet_list = [
    "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/hotpotqa_qwen_filtered_start_idx0_end_idx2500_seq16384/train.parquet",
    "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/musique_qwen_filtered_start_idx0_end_idx2500_seq16384/train.parquet",
    "/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/2wikimqa_qwen_filtered_start_idx0_end_idx2500_seq16384/train.parquet",
]
for parquet_path in no_dis_parquet_list:
    stats = analyze_prompt_tokens(parquet_path)
    pprint(f"{parquet_path} -> {stats}")

2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/hotpotqa_qwen_filtered_start_idx0_end_idx2500_seq16384/train.parquet '
 "-> {'max': 16279, 'min': 12058, 'avg': 14278.1976}")
2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/musique_qwen_filtered_start_idx0_end_idx2500_seq16384/train.parquet '
 "-> {'max': 16283, 'min': 12562, 'avg': 15144.9012}")
2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/2wikimqa_qwen_filtered_start_idx0_end_idx2500_seq16384/train.parquet '
 "-> {'max': 16274, 'min': 10727, 'avg': 14313.3924}")


In [15]:

dataset_prefix="/mnt/longcontext/models/siyuan"
other_parquet_list = [
    f"{dataset_prefix}/rl_datasets/rl_three/system/DAPO-Math-17k_2500/train.parquet",
    f"{dataset_prefix}/rl_datasets/rl_three/system/mathqa_choice_start_idx0_end_idx2500/train.parquet",
    f"{dataset_prefix}/rl_datasets/rl_three/system/book_20key_10k_512_start_index_0_end_index_512_seqlen16k/train.parquet",
    f"{dataset_prefix}/rl_datasets/rl_three/system/book_20value_10k_512_start_index_0_end_index_512_seqlen16k/train.parquet",
    f"{dataset_prefix}/rl_datasets/rl_three/system/sentence_needle_fix_500_start_index_0_end_index_500_seq_16384/train.parquet",
]

for parquet_path in other_parquet_list:
    stats = analyze_prompt_tokens(parquet_path)
    pprint(f"{parquet_path} -> {stats}")

2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/DAPO-Math-17k_2500/train.parquet '
 "-> {'max': 1014, 'min': 65, 'avg': 136.7232}")
2500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/mathqa_choice_start_idx0_end_idx2500/train.parquet '
 "-> {'max': 425, 'min': 40, 'avg': 85.0648}")
512
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/book_20key_10k_512_start_index_0_end_index_512_seqlen16k/train.parquet '
 "-> {'max': 17387, 'min': 12038, 'avg': 14772.046875}")
512
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/book_20value_10k_512_start_index_0_end_index_512_seqlen16k/train.parquet '
 "-> {'max': 17840, 'min': 11648, 'avg': 13671.439453125}")
500
('/mnt/longcontext/models/siyuan/rl_datasets/rl_three/system/sentence_needle_fix_500_start_index_0_end_index_500_seq_16384/train.parquet '
 "-> {'max': 16281, 'min': 16281, 'avg': 16281.0}")
